In [13]:
import requests, os,sys
from urllib.parse import quote
from bs4 import BeautifulSoup

import pandas as pd
query=quote('Dairy Manufacturing ERP ')
#user_input=input('Please type your keywords :  ')
#query=quote(str(user_input))
start=2011
finish=2021
pages=[]
for page_value in range(0,5,1): 
    url='https://scholar.google.com/scholar?start='+str(page_value) +'0&q=' + query  + '&hl=en&as_sdt=0,5&as_ylo='+str(start)+'&as_yhi='+str(finish)
    pages.append(url)
    

everything=[]
for url in pages: 
    headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
    proxies = {'http': os.getenv('HTTP_PROXY')}
    content = requests.get(url).text
    response=requests.get(url,headers=headers, proxies=proxies)

    page_contents = response.text
    doc = BeautifulSoup(page_contents,'html.parser')
    everything.append(doc)

titles=[]
links = []
cite_count = []
cited_by=[]
pub_info_list=[]
for doc in everything:
    
    paper_names_tag = doc.select('[data-lid]')
 
    for tag in paper_names_tag:
        titles_of_articles=tag.select('h3')[0].get_text()
        link=tag.select('a')[0]['href']
        links.append(link)
        publication_info = tag.select_one('.gs_a').get_text()
        titles.append (titles_of_articles) 
        citations=tag.select_one('#gs_res_ccl_mid .gs_nph+ a').get_text()
        cited_by.append(citations)
        pub_info_list.append(publication_info)
        

date=[]
for words in pub_info_list:
    numeric_filter = filter(str.isdigit, words)
    numeric_string = "".join(numeric_filter)
    date.append(numeric_string)  


years=[]

for values in date:
        
    if values[0:3]=='201' or  values[0:3]=='202'  or values[0:3]=='200':
        strings_of_years = values[0:4]
        years.append(strings_of_years)
    else:    
        strings_of_years = values[-4:]
        years.append(strings_of_years)

In [14]:
sort_years=[]
sort_titles=[]
sort_links=[]
for k in range(0,len(years)-1,1):
    if len(years[k])==4:
        sort_years.append(years[k])
        sort_links.append(links[k])
        sort_titles.append(titles[k])    
    
paper_dict_years = {
    'paper title ' : sort_titles,
    'url of paper' : sort_links,
    'date of publication': sort_years }
df = pd.DataFrame(paper_dict_years)
df.sort_values(by=['date of publication'], inplace=True, ascending=False)

#print(df)

                                         paper title   \
1   A REVIEW PAPER ABOUT THE APPLICATION OF MANUFA...   
40  Industry 4.0 and Digital Supply-Chain Manageme...   
28  Reduce waste, boost profits with process autom...   
34  Organizational performance through dairy suppl...   
38  [HTML][HTML] Evaluation of an eating time sens...   
23  Adoption of Enterprise Resources Planning (ERP...   
5   [HTML][HTML] A proposed framework model for da...   
4   [PDF][PDF] Using precision farming to improve ...   
41  Optimal production scheduling in the dairy ind...   
44  [PDF][PDF] Forecasting Techniques for Sales of...   
13  Optimising enterprise resource planning system...   
27  [HTML][HTML] A2A2 milk: Brazilian consumers' o...   
26  An overview of next-generation manufacturing e...   
9   [CITATION][C] EVALUATION OF EFFECTIVENESS OF E...   
12  [PDF][PDF] Factors Influencing Sustainable Ope...   
32  [PDF][PDF] Dynamic Pricing at retail market wi...   
2   [PDF][PDF] Implications of 

In [15]:
citations_in_words=[]
for cited_string in cited_by:
    word='Cited by'
    if word in cited_string:
        citations_in_words.append(cited_string)
    else:
        citations_in_words.append(0)


citations_integers=[]
for numbers in citations_in_words:
    if numbers==0:
        citations_integers.append(numbers)
    else:
        cite_filter = filter(str.isdigit, numbers)
        numeric_citations = "".join(cite_filter)
        cite_integers=int(numeric_citations)
        citations_integers.append(cite_integers)
 

paper_dict_citations = {
    'paper title ' : titles,
    'url of paper' : links,
    'citations': citations_integers }
df2 = pd.DataFrame(paper_dict_citations)
df2.sort_values(by=['citations'], inplace=True, ascending=False)

#print(df2)

                                         paper title   \
8   How do top managers support strategic informat...   
7   The Readiness of ERP Systems for the Factory o...   
12  Lean Six Sigma and environmental sustainabilit...   
9   Maintenance management and lean manufacturing ...   
28  An overview of next-generation manufacturing e...   
18  Measuring agri-food supply chain performance a...   
27  Benchmarking the interactions among performanc...   
37  A Delphi Examination of Inhibitors of The Effe...   
32     [PDF][PDF] Exploring ERP adoption cost factors   
33  [HTML][HTML] Genetic diversity of lactase pers...   
44       [PDF][PDF] ERP adoption cost factors in SMEs   
41  Using a dairy management information system to...   
43  Optimal production scheduling in the dairy ind...   
29  [HTML][HTML] A2A2 milk: Brazilian consumers' o...   
6   [HTML][HTML] A proposed framework model for da...   
47  Peeking at the ERP Decline stage: Japanese emp...   
15  Optimising enterprise resou

In [16]:
valid_links=[]
valid_names=[]

year_url_list = df['url of paper'].tolist()
year_title_list = df['paper title '].tolist()
cited_url_list=df2['url of paper'].tolist()
cited_title_list=df2['paper title '].tolist()

reverse_most_cited_url=cited_url_list[::-1]
reverse_most_cited_title=cited_title_list[::-1]

combined_url=year_url_list+reverse_most_cited_url
combined_title=year_title_list+reverse_most_cited_title
duplicate_url = list(dict.fromkeys(combined_url))
duplicate_title = list(dict.fromkeys(combined_title))

for s in range(0, len(duplicate_url)-1, 1):
    if duplicate_url[s].startswith('http://') or duplicate_url[s].startswith('https://') or duplicate_url[s].startswith('ftp://'):
        valid_links.append(duplicate_url[s])
        valid_names.append(duplicate_title[s])

pdf_url=[]
pdf_title=[]
unknown_format=[]
import requests
for v in range(0, len(valid_names)-1, 1):
    
    try:
        urls=valid_links[v]
        names=valid_names[v]
        new_name=''.join(e for e in names if (e.isalnum() or e.isspace()))
        r= requests.get(urls)
        content_type = r.headers.get('content-type')

        if 'application/pdf' in content_type:

            ext = '.pdf'
        elif 'text/html' in content_type:
            ext = '.html'
        else:
            ext = ''
            unknown_format.append(urls)
            

        if ext == '.pdf':
            pdf_url.append(urls)
            pdf_title.append(new_name)
        
    except Exception:
        pass
   
print(df)

                                         paper title   \
1   A REVIEW PAPER ABOUT THE APPLICATION OF MANUFA...   
40  Industry 4.0 and Digital Supply-Chain Manageme...   
28  Reduce waste, boost profits with process autom...   
34  Organizational performance through dairy suppl...   
38  [HTML][HTML] Evaluation of an eating time sens...   
23  Adoption of Enterprise Resources Planning (ERP...   
5   [HTML][HTML] A proposed framework model for da...   
4   [PDF][PDF] Using precision farming to improve ...   
41  Optimal production scheduling in the dairy ind...   
44  [PDF][PDF] Forecasting Techniques for Sales of...   
13  Optimising enterprise resource planning system...   
27  [HTML][HTML] A2A2 milk: Brazilian consumers' o...   
26  An overview of next-generation manufacturing e...   
9   [CITATION][C] EVALUATION OF EFFECTIVENESS OF E...   
12  [PDF][PDF] Factors Influencing Sustainable Ope...   
32  [PDF][PDF] Dynamic Pricing at retail market wi...   
2   [PDF][PDF] Implications of 

### Download Most Recent

In [11]:
count_downloads=[]
count=0
while count < len(pdf_url) and sum(count_downloads)<52: 

    name=pdf_title[count]
    url = pdf_url[count]
    filename = name+'.pdf'
    try:
        print("Creating the connection ...")
        r=requests.get(url, stream=True, timeout=5.0)

        if r.status_code != 200:
            print("Could not download the file '{}'\nError Code : {}\nReason : {}\n\n".format(
                        url, r.status_code, r.reason), file=sys.stderr) 
        else: 
            print("Saving the pdf file  :\n\"{}\" ...".format(filename))
            with open(filename, 'wb') as f:
                total_size = int(r.headers['Content-length'])
                print('file size : ' +str(total_size/1000.0) + ' kB')
                saved_size_pers = 0
                moversBy = 8192*100/total_size
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

                        saved_size_pers += moversBy
                        print("\r=>> %.2f%%" % (
                            saved_size_pers if saved_size_pers <= 100 else 100.0), end='', )


                print(end='\n\n')
                if total_size/1000.0>10.0:
                    count_downloads.append(1)
                    print('Number of PDFs downloaded so far : ' + str(sum(count_downloads)) )



    except requests.exceptions.ConnectionError:
        print("Connection Error")

    except requests.exceptions.HTTPError:
        print("HTTP error")


    except requests.exceptions.Timeout:
        print('Timeout error')

    except requests.exceptions.RequestException:
        print('Other errors')

    except Exception:
        print("==> Couldn't save : {}\\".format(filename))

        r.close()

    count=count+1
    print('number of iterations : ' + str(count))
    count_downloads=count_downloads


else:
    print('Downloading complete!')
    print(str(len(count_downloads)) + ' Most recent PDFs were downloaded')
    
    
print(df2)

Creating the connection ...
Saving the pdf file  :
"Adoption of Enterprise Resources Planning ERP in Indonesian Manufacturing Companies.pdf" ...
file size : 492.142 kB
=>> 100.00%

Number of PDFs downloaded so far : 1
number of iterations : 1
Creating the connection ...
Saving the pdf file  :
"PDFPDF Using precision farming to improve animal welfare.pdf" ...
file size : 648.145 kB
=>> 100.00%

Number of PDFs downloaded so far : 2
number of iterations : 2
Creating the connection ...
Saving the pdf file  :
"PDFPDF Forecasting Techniques for Sales of Spare Parts.pdf" ...
file size : 323.758 kB
=>> 100.00%

Number of PDFs downloaded so far : 3
number of iterations : 3
Creating the connection ...
Saving the pdf file  :
"Optimising enterprise resource planning system to leverage a firms absorptive and adaptive capabilities.pdf" ...
==> Couldn't save : Optimising enterprise resource planning system to leverage a firms absorptive and adaptive capabilities.pdf\
number of iterations : 4
Creating

### Download Most Cited

In [12]:
count_downloads=[]
count=0
reverse_title= pdf_title[::-1]
reverse_url= pdf_url[::-1]

while count < len(reverse_url) and sum(count_downloads)<52.0: 

    name=reverse_title[count]
    url =reverse_url[count]
    filename = name+'.pdf'
    try:
        print("Creating the connection ...")
        r=requests.get(url, stream=True, timeout=5.0)

        if r.status_code != 200:
            print("Could not download the file '{}'\nError Code : {}\nReason : {}\n\n".format(
                        url, r.status_code, r.reason), file=sys.stderr) 
        else: 
            print("Saving the pdf file  :\n\"{}\" ...".format(filename))
            with open(filename, 'wb') as f:
                total_size = int(r.headers['Content-length'])
                print('file size : ' +str(total_size/1000.0) + ' kB')
                saved_size_pers = 0
                moversBy = 8192*100/total_size
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

                        saved_size_pers += moversBy
                        print("\r=>> %.2f%%" % (
                            saved_size_pers if saved_size_pers <= 100 else 100.0), end='', )




                print(end='\n\n')
                if total_size/1000.0>10.0:
                    count_downloads.append(1)
                    print('Number of PDFs downloaded so far : ' + str(sum(count_downloads)) )



    except requests.exceptions.ConnectionError:
        print("Connection Error")

    except requests.exceptions.HTTPError:
        print("HTTP error")


    except requests.exceptions.Timeout:
        print('Timeout error')

    except requests.exceptions.RequestException:
        print('Other errors')

    except Exception:
        print("==> Couldn't save : {}\\".format(filename))

        r.close()

    count=count+1
    print('number of iterations : ' + str(count))
    count_downloads=count_downloads


else:
    print('Downloading complete!')
    print(str(len(count_downloads)) + ' Most cited PDFs were downloaded') 

Creating the connection ...
Saving the pdf file  :
"PDFPDF Evaluation of Scope of Using the Information Systems Depending on Sector of Agribusiness.pdf" ...
file size : 810.086 kB
=>> 100.00%

Number of PDFs downloaded so far : 1
number of iterations : 1
Creating the connection ...
Saving the pdf file  :
"PDFPDF ERP adoption cost factors in SMEs.pdf" ...
file size : 162.509 kB
=>> 100.00%

Number of PDFs downloaded so far : 2
number of iterations : 2
Creating the connection ...
Saving the pdf file  :
"PDFPDF Exploring ERP adoption cost factors.pdf" ...
file size : 544.83 kB
=>> 100.00%

Number of PDFs downloaded so far : 3
number of iterations : 3
Creating the connection ...
Saving the pdf file  :
"A Delphi Examination of Inhibitors of The Effective use of Process Industry Enterprise Resource Planning Erp Systems A Case Study of New Zealand s Process .pdf" ...
file size : 413.365 kB
=>> 100.00%

Number of PDFs downloaded so far : 4
number of iterations : 4
Creating the connection ...
S